In [1]:
import pandas as pd
import numpy as np
import sklearn
import scipy
from scipy import spatial
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
data = pd.read_csv(r"C:\Users\SAN\Downloads\kmeans_data\data.csv")
labels = pd.read_csv(r"C:\Users\SAN\Downloads\kmeans_data\label.csv",names=['label'],header=None)

In [3]:
data.head(10)

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [5]:
#dividing the data for both testing and training
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [6]:
class KMeans:
    
    def cal_sse_value(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            # np.sum()
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def start_centroid(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jacr_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.start_centroid(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jacr_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict
    

In [7]:
def forecast_clust_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [8]:
def jacr_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [9]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jacr_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [10]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  27930.6358490566
Tolerance for the Iteration  1 :  5470.064311259095
Tolerance for the Iteration  2 :  3140.601809954751
Tolerance for the Iteration  3 :  2575.4996408309894
Tolerance for the Iteration  4 :  2448.7049198549203
Tolerance for the Iteration  5 :  1918.2883685997908
Tolerance for the Iteration  6 :  1067.5724038603687
Tolerance for the Iteration  7 :  636.027371353948
Tolerance for the Iteration  8 :  344.3472906454565
Tolerance for the Iteration  9 :  279.9538291457287
Tolerance for the Iteration  10 :  238.40161033797204
Tolerance for the Iteration  11 :  181.85198876531436
Tolerance for the Iteration  12 :  135.24984328451768
Tolerance for the Iteration  13 :  136.43384444107093
Tolerance for the Iteration  14 :  93.66061581267482
Tolerance for the Iteration  15 :  78.95390762684917
Tolerance for the Iteration  16 :  92.13577462643778
Tolerance for the Iteration  17 :  85.7653276955603
Tolerance for the Iteration  18 :  76.0037417362999

In [11]:
Euclidean_SSE = model1.cal_sse_value(centroids1,clusters1,data)

In [12]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25322297989.80762


In [13]:
cluster_labels1 = forecast_clust_labels(centroids1,clusters1,labels)
cluster_labels1

array([9, 2, 4, 3, 5, 6, 7, 1, 0, 0])

In [14]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.09125

In [15]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = model2.cal_sse_value(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  38195.5404040404
Tolerance for the Iteration  1 :  3540.602168473728
Tolerance for the Iteration  2 :  1730.3915694783752
Tolerance for the Iteration  3 :  1556.2286228622863
Tolerance for the Iteration  4 :  1181.7879739193431
Tolerance for the Iteration  5 :  949.2851970476236
Tolerance for the Iteration  6 :  1295.2449623533782
Tolerance for the Iteration  7 :  1117.3542903029797
Tolerance for the Iteration  8 :  1148.4447868489406
Tolerance for the Iteration  9 :  967.0712897376693
Tolerance for the Iteration  10 :  1103.5683976934276
Tolerance for the Iteration  11 :  1267.092373040703
Tolerance for the Iteration  12 :  1055.5544066921593
Tolerance for the Iteration  13 :  0.0


In [16]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [17]:
cluster_labels2 = forecast_clust_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 2, 0, 7, 8, 4, 2, 1, 0, 3])

In [18]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [19]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  34456.63884342607
Tolerance for the Iteration  1 :  5300.491851778976
Tolerance for the Iteration  2 :  3546.5668821978306
Tolerance for the Iteration  3 :  2608.298969320813
Tolerance for the Iteration  4 :  2385.602458365116
Tolerance for the Iteration  5 :  2871.5684353395263
Tolerance for the Iteration  6 :  2698.5355073525125
Tolerance for the Iteration  7 :  1915.5867486509333
Tolerance for the Iteration  8 :  1280.9851575327361
Tolerance for the Iteration  9 :  840.8274885352037
Tolerance for the Iteration  10 :  982.7842537622826
Tolerance for the Iteration  11 :  806.7785598120881
Tolerance for the Iteration  12 :  554.6098783419911
Tolerance for the Iteration  13 :  420.28444104914684
Tolerance for the Iteration  14 :  326.3644744483161
Tolerance for the Iteration  15 :  206.98752378631457
Tolerance for the Iteration  16 :  239.33025133987033
Tolerance for the Iteration  17 :  187.49723456399718
Tolerance for the Iteration  18 :  137.03095317

In [20]:
Cosine_SSE = model3.cal_sse_value(centroids3,clusters3,data)

In [26]:
print("Euclidean Sum of Squares Error:",Euclidean_SSE)
print("Jacard Sum of Squares Error:",Jaccard_SSE)
print("Cosine Sum of Squares Error :",Cosine_SSE)

Euclidean Sum of Squares Error: 25322297989.80762
Jacard Sum of Squares Error: 34361687572.938736
Cosine Sum of Squares Error : 25553951287.7575


In [22]:
cluster_labels3 = forecast_clust_labels(centroids3,clusters3,labels)
cluster_labels3

array([0, 7, 1, 0, 2, 4, 3, 3, 8, 9])

In [23]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

0.07375

In [27]:
print("Euclidean-K-means accuracy:",Accuracy_Euclidean)
print("Jacard-K-means accuracy:",Accuracy_Jaccard)
print("Cosine-K-means accuracy :",Accuracy_Cosine)

Euclidean-K-means accuracy: 0.09125
Jacard-K-means accuracy: 0.1075
Cosine-K-means accuracy : 0.07375
